In [92]:
# basic Library
import sys
import re
from collections import Counter
import pickle

# basic ML library
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# konlpy library
from konlpy.tag import Okt

# graph korean expression
plt.rc('font', family='Malgun Gothic')

In [93]:
# pik = pd.read_csv("piks_pik.csv")
# cat = pd.read_csv("piks_category.csv")
# link = pd.read_csv("linkhub_link.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_17956\386778698.py:3: DtypeWarning: Columns (11,20) have mixed types. Specify dtype option on import or set low_memory=False.
  link = pd.read_csv("linkhub_link.csv")


In [78]:
# check_list = ["SNS 마케팅, 이거 하나로 끝내기", "제주 여행을 위한 맛집 핫플 브이로그 모음", "투자가 처음이신가요? 어린이를 위한 투자 정보"]
# pik2 = pik[pik["title"] == check_list[0]]

In [95]:
link = pd.read_csv("pik_title_unique.csv")

In [96]:
## pikurate bot id Load pickle
with open("artificial_user_list","rb") as fr:
    pik_bot_id = pickle.load(fr)
print(pik_bot_id)

[1576, 4558, 1337, 3448, 3464, 3466, 4864, 4913, 4761, 4918, 4817, 4763, 4856, 4846, 4854, 5213, 4979, 4760, 6359, 6324, 6356, 6384, 6230, 6276, 6257, 6232, 6223, 6221, 6299, 6233, 6224, 6291, 6235, 6255, 6269, 6238, 6220, 6259, 6314, 6250, 6287, 6239, 6244, 6241, 6289, 6242, 6260, 6275]


In [98]:
# pikurate bot del
new_link = link[~link['user_id'].isin(pik_bot_id)]
print(f"피큐봇 제거 전 Link 데이터 개수 : {pik_title_unique.shape[0]}개")
print(f"피큐봇 제거 후 Link 데이터 개수 : {new_link.shape[0]}개")

피큐봇 제거 전 Link 데이터 개수 : 1872개
피큐봇 제거 후 Link 데이터 개수 : 1872개


In [100]:
def duplicate_pik_title (dataframe, col):
    dup_pik_list = ["SNS 마케팅, 이거 하나로 끝내기", "제주 여행을 위한 맛집 핫플 브이로그 모음", "투자가 처음이신가요? 어린이를 위한 투자 정보"]
    for dup_pik in dup_pik_list:
        dataframe = dataframe[dataframe[col] != dup_pik]
    return dataframe

In [102]:
# duplicate_pik_title (new_link, 'pik_title')

,Unnamed: 0,user_id,pik_id,pik_title,category_id,cat_title,link_id,link_title,topic_label
0,0,3,94,portland maine trip,405,Trip,1379,visit portland maine travel planning official ...,3
1,1,13,71,how to find the best movie,291,movie DB,873,the 50 best movies on netflix right now,14
2,3,13,190,marketing to nordic countries,761,12,903,youtube,13
3,4,13,226,places to visit on bali,768,beach,2097,kuta wikipedia,3
4,7,17,31,data,109,Generalaegrhaerha4h,233,coreware labs,2
...,...,...,...,...,...,...,...,...,...
1867,37403,10483,27913,questiongenerator,91077,ai,620016,question generation pororo,2
1868,37411,10544,28051,ios root,91449,인증서 조회,621286,need list of root certs on ios apple developer...,1
1869,37419,10555,28079,jpa,91529,미분류,621523,spring data jpa reference documentation,1
1870,37444,10564,28169,ios,91776,개발,623206,ios,1


In [84]:
new_link.columns

Index(['id', 'created', 'modified', 'title', 'description', 'url',
       'category_id', 'picture_id', 'user_id', 'default_logo', 'position',
       'search_vector', 'old_id', 'memo', 'is_deleted', 'is_draft', 'platform',
       'curating_point', 'next_obj', 'previous_obj', 'imported'],
      dtype='object')

In [85]:
# stop_words
stop_words = {"네이버", "블로그", "방법", "홈페이지", "가지", "만들기", "플루", "우리"}

In [103]:
# regular expression
def ko_preprocess(document : str) -> str:
    # ko word sub
    pattern = r'[^가-힣ㄱ-ㅎㅏ-ㅣ]' 
    document = re.sub(pattern, ' ', document)
    # remove empty space
    document = document.strip()
    # make empty spcae size as only one
    document = ' '.join(document.split())
    return document

In [104]:
# konlpy pos Extraction
def konlpy_pos (text):
    # konlpy twitter model
    okt = Okt()
    okt_pos = okt.pos(text)
    new_pos = []
    # nouns word classify
    for word, pos in okt_pos:
        if pos == "Noun":
            # stopwords 
            if word in stop_words: 
                continue
            # length 2 or more word
            if len(word) >= 2:
                new_pos.append(word)
    return new_pos

In [105]:
# word Counter
def word_top_counter (text, ntags=500):
    count = Counter(text)
    return { n : c  for n, c in count.most_common(ntags)}

In [106]:
# top frequency graph
def frequency_graph (top_word):
    plt.figure(figsize=(15,10))
    sorted_keys = sorted(top_word, key=top_word.get, reverse=True)
    sorted_values = sorted(top_word.values(), reverse=True)
    plt.bar(range(len(top_word)), sorted_values, align="center", linewidth=0.8)
    plt.xticks(range(len(top_word)), list(sorted_keys), rotation="75")
    plt.xlabel("키워드")
    plt.ylabel("빈도수")
    plt.grid(True)
    plt.show()

In [117]:
# ko nlp prepreocess
link_title = new_link.loc[:, "title"]
ko_word = [ko_preprocess(str(n)) for n in link_title]
ko_word[:5]

KeyError: 'title'

In [109]:
# konlpy class
ko_ex_word = []
for kw in ko_word:
    ko_ex_word.extend(konlpy_pos(kw))

In [110]:
# 500개 counter
top_cnt = word_top_counter(ko_ex_word)
top_cnt

{}

In [ ]:
# # top_500 csv 저장 
# ko_top_500 = pd.DataFrame(top_cnt.items(), columns=["text", "count"])
# ko_top_500.to_csv("NLP/ko_top_500.csv")

In [ ]:
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
import numpy as np
from PIL import *

In [ ]:
import stylecloud
font_path = 'c:\Windows\Fonts\malgun.ttf'
# 스타일 설정
stylecloud.gen_stylecloud(top_cnt, icon_name='fas fa-cloud', 
                          font_path = 'c:\Windows\Fonts\malgun.ttf',
                          palette='colorbrewer.diverging.Spectral_11', 
                          background_color='black', 
                          gradient='horizontal')

# 클라우드 그림 생성
stylecloud.gen_stylecloud(top_cnt, icon_name='fas fa-cloud', 
                          font_path = 'c:\Windows\Fonts\malgun.ttf',
                          palette='colorbrewer.diverging.Spectral_11', 
                          background_color='black', 
                          gradient='horizontal', output_name='hello_world.png')

